## SECOND EXPERIMENT

Для второго эксперимента мы использовали подход, представленный по этой [ссылке](https://colmap.github.io/faq.html#reconstruct-sparse-dense-model-from-known-camera-poses) в документации COLMAP. 

Суть данного эксперимента заключается в том, что мы будем использовать возможности аппарата PixSfM, а именно featuremetric KA до построения SfM и feature-reference BA после построения SfM.


### Reconstruct sparse/dense model from known camera poses 

| COLMAP | PixSfM | 
| --- | --- |
| 1)  Создать три файла cameras.txt, images.txt, point3D.txt в одной папке. | 1) Создать три файла cameras.txt, images.txt, point3D.txt в одной папке |
| 2) Создать пустой файл points3D.txt | 2) Создать пустой файл points3D.txt |
| 3) В файле images.txt каждая вторая строка для изображения должна быть пустой, cameras.txt должен иметь всю инфу. | 3) В файле images.txt каждая вторая строка для изображения должна быть пустой, cameras.txt должен иметь всю инфу. |
| 4) Заполняем БД инфой о камерах и изображениях. | 4) Заполняем БД инфой о камерах и изображениях.|
| 5) Выполняем feature extraction при помощи COLMAP и записываем фичи в БД. | 5)  Выполняем feature extraction при помощи **hloc**.|
| 6) Выполняем exhausting matching. | 6) Выполняем exhausting matching при помощи **hloc**.|
| | 7) Выполняем **Featuremetric KA** (Keypoint Adjustment), запись улучшенных features в БД. |
| 7) Построение SfM (этап triangulation).| 8) Построение SfM (этап triangulation).|
|  | 9) Выполняем **feature-reference BA** (Bundle Adjustment).|


# Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars

from pathlib import Path
import pycolmap
import sys

sys.path.append("/workspace/pixel-perfect-sfm/")
sys.path.append("/workspace/pixel-perfect-sfm/Hierarchical-Localization")

from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from hloc.utils.read_write_model import  write_next_bytes, Point3D, Image, read_images_text, \
        read_points3D_binary, write_points3D_binary, write_images_binary, read_images_binary, \
        read_cameras_text

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect

from utils import modified_write_points3D_text

import numpy as np

import open3d as o3d
assert o3d.__version__ == '0.15.2', 'The version 0.15.2 is required!'

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

# Setup

В **object_name** необходимо задать имя объекта, над которым вы хотите провести эксперимент.

**check_for_calibrated_images** - булевая переменная, по которой мы выбираем какие поз камер использовать (менее точные (True) или точные (False))

**delete_previous_output** - если True, то удаляет все предыдущие файлы в папке outputs. Использовать супер осторожно.

**has_cache** - если True, то у Вас уже существует файл с feature maps и он сохранен в папке cache_init. Это файл с feature maps Вы получаете только тогда, когда вы уже сделали featuremetric KA или BA для одного из ваших экспериментов.

**show_visualization** - если True, то показывает визуализацию результата эксперимента (3d pointcloud, задектированные keypoints (features) и final reprojections для какого-то изображения).

In [ ]:
object_name = 'dragon'

check_for_calibrated_images = False
delete_previous_output = False

has_cache = True
show_visualization = False

**images_init** - путь к файлу images.txt с известными позами камер (каждая вторая строка пустая)

**calibrated_images_init** - путь к файлу images.txt c известнами позами камер (но менее точные)

**cameras_init** - путь к файлу cameras.txt

**images** - путь к папке с изображениями для реконструкции

**outputs** - путь к папке со всеми результатами

**cache_init** - путь к кэш-файлу, его мы получаем во время того, когда делаем KA или BA. В этот файле хранятся featuremaps после  dense feature extraction. В среднем на одну картинку размером 2368х1952 уходит 3 минуты. Этот файл вообще нельзя трогать, поэтому мы копируем его в папку outputs для своего эксперимента и продолжаем работу.

**cache_path** - тот же файл, что cache_init, с которым мы теперь будем работать во время эксперимента.

**sfm_pairs** - файл с названиями пар изображений на каждой строке

**features** - файл с features для каждой картинки, извлеченными при помощи feature_conf

**matches** - файл с matches для каждой пары картинок, извлеченными при помощи matcher_conf

In [ ]:
root = Path('/workspace')

images_init = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/images.txt'
calibrated_images_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/images.txt' # менее точные

cameras_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/cameras.txt'

images = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/undistorted/ambient@best'

outputs = root / f'pixel-perfect-sfm/outputs/{object_name}_exp2/'

if delete_previous_output:
    !rm -rf $outputs 
    
outputs.mkdir(parents=True, exist_ok=True)    
    
if has_cache:
    cache_init = root / f'pixel-perfect-sfm/outputs/caches/{object_name}/s2dnet_featuremaps_sparse.h5'
    !cp -r $cache_init $outputs
    cache_path = outputs / 's2dnet_featuremaps_sparse.h5'    

sfm_pairs = outputs / 'pairs-sfm.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

**exp2_dir** - папка, в которой будет сохранен результат второго эксперимента.

In [ ]:
exp2_dir = outputs / "exp2"
exp2_dir.mkdir(parents=True, exist_ok=True)

if check_for_calibrated_images:
    images_init = calibrated_images_init
    
    exp2_dir = outputs / "calibrated/exp2"
    exp2_dir.mkdir(parents=True, exist_ok=True)

# 3D mapping and refinement

Здесь описаны возможности для настройки [**extract_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/extract_features.py#L21)

Здесь описаны возможности для настройки [**match_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/match_features.py#L17)

Здесь описан пайплайн того, как можно использовать свои кастомные [**local features**, **matcher**, **image retrieval**](https://github.com/cvg/Hierarchical-Localization/tree/91f40bfd765add3b59ba7376f8579d8829f7fa78#using-your-own-local-features-or-matcher).


In [ ]:
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

Здесь мы проверяем какие изображения мы будем использовать для реконструкции.

In [ ]:
references = [str(p.relative_to(images)) for p in images.iterdir()]
print(len(references), "mapping images")
plot_images([read_image(images / r) for r in references[:4]], dpi=50)

**extract_features** - данная функция получает на вход *feature_conf*, *images* (путь к папке с изображениями), *image_list* (список тех изображений, которые вы хотите использовать для feature exctraction), *feature_path* (путь к файлу, где будет сохранен результат). На выходе получаем файл (**features**) с извлеченными features. Если **features** существует, то пропускается.

**pairs_from_exhaustive** - данная функция получает на вход *sfm_pairs* (путь к файлу, где будет сохранен результат), *image_list* (список тех изображений, при помощи которых вы сделаете exhaustive pairs.) На выходе получаем файл (**sfm_pairs**) с парами изображений.  Если **sfm_pairs** существует, то пропускается.

**match_features** - данная функция получает на вход *matcher_conf*, *sfm_pairs* (путь к файлу, где хранятся пары изображений после exhaustive pairing), *features* (путь к файлу, где хранятся извлеченный features для каждого изображения), *matches* (путь к файлу, где хранятся matches для каждой пары изображения). На выходе получаем файл (**matches**) с matches для каждой пары изображений . Если **match_features** существует, то пропускается.



In [ ]:
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/features.h5 $outputs
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/matches.h5 $outputs
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/pairs-sfm.txt $outputs

Ниже клетка может выполняться от получаса до часа (в зависимости от нагруженности).

Features extraction - 1 минута.

Features matching - 35-50 минут.

In [ ]:
extract_features.main(feature_conf, images, image_list=references, feature_path=features) 
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches); 

Копируем файлы images.txt, cameras.txt в папку для второго эксперимента. В этой папке мы должны иметь три файла (images.txt, cameras.txt и пустой файл point3d.txt).

In [ ]:
!cp -r $images_init $exp2_dir
print("images.txt copied!")

!cp -r $cameras_init $exp2_dir
print("cameras.txt copied!")

!touch $exp2_dir/points3D.txt
print("points3D.txt created!")

!ls $exp2_dir

Смотрим при помощи **pycolmap** информацию о полученной на данной момент реконструкции.

In [ ]:
check_model = pycolmap.Reconstruction(exp2_dir)
print(check_model.summary())

**Featuremetric KA (Keypoint Adjustment)**

В конфиге conf_KA расписана необходимая информация для того, чтобы сделать KA. В этом конфиге также указано, что вы хотите использовать cache, который находится в пути cache_path.

**keypoints_path** -  здесь будут сохранены обновленная информация по уже имеющимся keypoints (features). После KA тут уже будут сохранены новые features.

Здесь описано как можно настроить конфигурацию для КА. https://github.com/cvg/pixel-perfect-sfm#detailed-configuration

 КА выполняется ниже за 3-4 минуты.

In [ ]:
from pixsfm.refine_hloc import PixSfM
from pixsfm.refine_colmap import PixSfM as PixSfM_ba

# running Keypoint Adjustment
conf_KA = {
        "dense_features": {
                "use_cache": True,
        },
        "KA": {
            "dense_features": {'use_cache': True}, 
            "split_in_subproblems": True,
            "max_kps_per_problem": 1000,  
        },
}

if not has_cache:
    conf_KA.update({
        "dense_features": {
            "use_cache": True,
            "sparse" : True,
            "dtype" : "half",
            "overwrite_cache": True,
            "load_cache_on_init": False,
            "patch_size": 8,
            "cache_format": "chunked"
        }

    })

refiner = PixSfM(conf=conf_KA)

keypoints_path = exp2_dir / "refined_keypoints.h5"

keypoints, ka_data, feature_manager = refiner.refine_keypoints(
    output_path = keypoints_path,
    image_dir = images,
    features_path = features,
    pairs_path = sfm_pairs,
    matches_path = matches,
    cache_path = cache_path,
)

if not has_cache:    
    caches = root / f'pixel-perfect-sfm/outputs/caches/exp2/{object_name}'
    caches.mkdir(parents=True, exist_ok=True)
    cache_path = outputs / 's2dnet_featuremaps_sparse.h5'
    !cp -r $cache_path $caches 

Импортируем **hloc**. Она нам поможет создать БД, заполнить БД, сделать геометрическую верификацию, сделать триангуляцию.

In [ ]:
# https://github.com/cvg/pixel-perfect-sfm/blob/main/pixsfm/refine_hloc.py

try:
    import hloc
except ImportError:
    print("Could not import hloc.")
    hloc = None

1) Создание БД

2) Импорт камер, картинок, улучшенных features, matches

3) Запись реконструкции до FMBA в папку.

Ниже клетка выполняется за 1 минуту.

In [ ]:
hloc_path = exp2_dir / 'hloc'
hloc_path.mkdir(parents=True, exist_ok=True)

database_path = hloc_path / 'database.db' 
reference = pycolmap.Reconstruction(exp2_dir)   

images_txt_path = exp2_dir / 'images.txt'
images_dict = read_images_text(images_txt_path)
        
# Here I changed code and in database we have data about camera extrinsics    
image_ids = hloc.triangulation.create_db_from_model(reference, 
                                                    database_path, 
                                                    images_dict)

#Importing features into database -> keypoints table 
hloc.triangulation.import_features(image_ids, 
                                   database_path, 
                                   keypoints_path)

#Importing matches into database -> matches table
skip_geometric_verification = False
hloc.triangulation.import_matches(image_ids, 
                                  database_path, 
                                  sfm_pairs, 
                                  matches,
                                  min_match_score=None, 
                                  skip_geometric_verification=skip_geometric_verification)

verbose, estimate_two_view_geometries = True, False

if not skip_geometric_verification:
        if estimate_two_view_geometries:
            hloc.triangulation.estimation_and_geometric_verification(database_path, 
                                                                     sfm_pairs, 
                                                                     verbose)
        else:
            # We are doing this part to add data to two_view_geometries table
            hloc.triangulation.geometric_verification(
                image_ids, 
                reference, 
                database_path, 
                keypoints_path, 
                sfm_pairs, 
                matches)
            
reconstruction = hloc.triangulation.run_triangulation(hloc_path, 
                                                      database_path, 
                                                      images, 
                                                      reference, 
                                                      verbose)

print(reconstruction.summary())  

# Saving result to a folder
reconstruction.write(str(hloc_path))

Проверяем те же ли параметры камер получились.

In [ ]:
# compare images old and new, compare cameras old and new
images_old = read_images_text(exp2_dir / 'images.txt')
images_new = read_images_binary(hloc_path / 'images.bin')

for i in range(1, 101):
    assert np.array_equal(images_old[i].tvec, images_new[i].tvec) == True
    assert np.allclose(np.array(images_old[i].qvec, dtype=np.float32), 
                          np.array(images_new[i].qvec, dtype=np.float32)) == True

**Featuremetric BA (Bundle Adjustment)**

В конфиге conf_BA расписана необходимая информация для того, чтобы сделать BA. В этом конфиге также указано, что вы хотите использовать cache, который находится в пути cache_path.

Здесь описано как настроить конфигурацию для ВА. https://github.com/cvg/pixel-perfect-sfm#detailed-configuration

Здесь ВА выполняется за 1.5-2 минуты.

In [ ]:
# running featuremetric BA
conf_BA = {
        "dense_features": {
                "use_cache": True,
        },
        
        "BA": {
            "dense_features": {'use_cache': True}, 
            "apply": True
        }
}

refiner = PixSfM_ba(conf=conf_BA)

reconstruction, ba_data, feature_manager2 = refiner.refine_reconstruction(
    output_path = exp2_dir / 'hloc/model',
    input_path = exp2_dir / 'hloc',
    image_dir = images,
    cache_path = cache_path,
)

print(reconstruction.summary())

Перевод модели в формат TXT.

In [ ]:
!mkdir -p $exp2_dir/hloc/model/model_txt/

!colmap model_converter \
    --input_path $exp2_dir/hloc/model \
    --output_path $exp2_dir/hloc/model/model_txt/\
    --output_type TXT

# Visualization

In [ ]:
fig3d = init_figure()

args = dict(max_reproj_error=3.0, 
            min_track_length=2, 
            cs=0.01) #camera size
plot_reconstruction(fig3d, reconstruction, 
                    color='rgba(0, 255, 0, 0.5)', 
                    name="refined", **args)
if show_visualization:
    fig3d.show()

In [ ]:
refined = reconstruction

img = refined.images[refined.reg_image_ids()[0]]
cam = refined.cameras[img.camera_id]

fig = init_image(images / img.name)    

plot_points2D(fig, [p2D.xy for p2D in img.points2D if p2D.has_point3D()])
plot_points2D(fig, cam.world_to_image(img.project(refined)), color='rgba(255, 0, 0, 0.5)')

if show_visualization:
    fig.show()